In [ ]:
!pip install datasets



In [ ]:
from datasets import load_dataset

data = load_dataset("rotten_tomatoes")
data

In [ ]:
data["train"][0,-1]

In [ ]:
from transformers import pipeline

model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe = pipeline(model = model_path, tokenizer= model_path, return_all_scores=True, device ="cuda:0")


In [ ]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

y_pred= []
for output in tqdm(pipe(KeyDataset(data["test"],"text")), total =len(data["test"])):
  negative_score = output[0]["score"]
  positive_score = output[2]["score"]
  assignemnt = np.argmax([negative_score, positive_score])
  y_pred.append(assignemnt)



In [ ]:
from sklearn.metrics import classification_report

def evaluate_per(y_true, y_pred):
  performance = classification_report(y_true, y_pred, target_names = ["Negative Review", "Positive Review"])
  print(performance)

evaluate_per(data["test"]["label"], y_pred)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
train_embeddings = model.encode(data["train"]["text"], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state =42)
clf.fit(train_embeddings, data["train"]["label"])

In [ ]:
y_pred = clf.predict(test_embeddings)
evaluate_per(data["test"]["label"], y_pred)

In [ ]:
label_embeddigs = model.encode(["A Negative review", "A positive review"])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix =cosine_similarity(test_embeddings, label_embeddigs)
y_pred = np.argmax(sim_matrix, axis=1)
evaluate_per(data["test"]["label"], y_pred)

In [ ]:
pipe = pipeline("text2text-generation", model = "google/flan-t5-small", device ="cuda:0")


In [ ]:
prompt ="is the following sentence positive or negative?"
data = data.map(lambda example:{"t5":prompt + example["text"]})
data

In [ ]:
y_pred= []
for output in tqdm(pipe(KeyDataset(data["test"], "t5")),total = len(data["test"])):
  text = output[0]["generated_text"]
  y_pred.append(0 if text == "negative" else 1)
evaluate_per(data["test"]["label"], y_pred)
